In [ ]:
# 하루 치의 최종 예측값 뽑는 파일

In [1]:
from flask import Flask, request, jsonify
from bs4 import BeautifulSoup
import requests
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import google.generativeai as genai
import json
import re
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import os
from typing import List
import tensorflow as tf, joblib, numpy as np
from CNN_PY import cnn_model
from LSTM_PY import lstm_model

c:\Users\pyeon\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
symbol = "MMM"
today = datetime.today()
print(f"today date : {today.strftime("%Y-%m-%d")}")

start_date = (today - timedelta(days=1)).strftime("%Y-%m-%d")
end_date = (today - timedelta(days=0)).strftime("%Y-%m-%d")
print(f"start date : {start_date}")
print(f"end   date : {end_date}\n")

# start <= 날짜 < end 조건임
data = yf.download(symbol, start=start_date, end=end_date, auto_adjust=True)
print(data.tail(10))

'''
yf.download: 

start: str
Download start date string (YYYY-MM-DD) or _datetime, inclusive.
Default is 99 years ago
E.g. for start="2020-01-01", the first data point will be on "2020-01-01"

end: str
Download end date string (YYYY-MM-DD) or _datetime, exclusive.
Default is now
E.g. for end="2023-01-01", the last data point will be on "2022-12-31"
'''

today date : 2025-07-01
start date : 2025-06-30
end   date : 2025-07-01



[*********************100%***********************]  1 of 1 completed

Price            Close        High    Low        Open   Volume
Ticker             MMM         MMM    MMM         MMM      MMM
Date                                                          
2025-06-30  152.240005  152.460007  151.0  152.279999  2302800


'\nyf.download: \n\nstart: str\nDownload start date string (YYYY-MM-DD) or _datetime, inclusive.\nDefault is 99 years ago\nE.g. for start="2020-01-01", the first data point will be on "2020-01-01"\n\nend: str\nDownload end date string (YYYY-MM-DD) or _datetime, exclusive.\nDefault is now\nE.g. for end="2023-01-01", the last data point will be on "2022-12-31"\n'

In [3]:
import csv
import os
from datetime import datetime

symbol= "MMM"
target_date= target_date = datetime.today().strftime("%Y-%m-%d")
print(f"target_date : {target_date}")


from_date = datetime.strptime(target_date, "%Y-%m-%d")
print(f"from_date : {from_date}")
start_dt = from_date - timedelta(days=150)
print(f"start_dt : {start_dt}")

df = yf.download(symbol, start=start_dt.strftime("%Y-%m-%d"), end=from_date.strftime("%Y-%m-%d"))
print(df.tail(10))

target_date : 2025-07-01
from_date : 2025-07-01 00:00:00
start_dt : 2025-02-01 00:00:00
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open   Volume
Ticker             MMM         MMM         MMM         MMM      MMM
Date                                                               
2025-06-16  145.070007  145.389999  143.509995  143.679993  1681300
2025-06-17  142.210007  144.880005  141.970001  144.059998  1739900
2025-06-18  142.509995  143.669998  141.570007  142.130005  2629100
2025-06-20  144.029999  144.050003  141.820007  143.309998  5566700
2025-06-23  147.600006  147.720001  143.350006  144.389999  2904700
2025-06-24  149.869995  150.289993  148.009995  148.639999  2700500
2025-06-25  147.750000  149.860001  147.119995  149.699997  2378000
2025-06-26  150.759995  151.110001  147.309998  147.690002  2740100
2025-06-27  152.020004  152.869995  150.559998  151.139999  3938100
2025-06-30  152.240005  152.460007  151.000000  152.279999  2302800


In [4]:
import yfinance as yf
from datetime import datetime, timedelta

today = datetime.today().date()  # 2025-06-24
print(today)
start = today - timedelta(days=6)  # 2025-06-18
print(start)

symbol = "AAPL"
data = yf.Ticker(symbol).history(start=start, end=today)

print(data)
print("\n최근 종가:", data['Close'].iloc[-1])


2025-07-01
2025-06-25
                                 Open        High         Low       Close  \
Date                                                                        
2025-06-25 00:00:00-04:00  201.449997  203.669998  200.619995  201.559998   
2025-06-26 00:00:00-04:00  201.429993  202.639999  199.460007  201.000000   
2025-06-27 00:00:00-04:00  201.889999  203.220001  200.000000  201.080002   
2025-06-30 00:00:00-04:00  202.009995  207.389999  199.259995  205.169998   

                             Volume  Dividends  Stock Splits  
Date                                                          
2025-06-25 00:00:00-04:00  39525700        0.0           0.0  
2025-06-26 00:00:00-04:00  50799100        0.0           0.0  
2025-06-27 00:00:00-04:00  73188600        0.0           0.0  
2025-06-30 00:00:00-04:00  91612700        0.0           0.0  

최근 종가: 205.1699981689453


In [5]:
# start 빼고 end만 넣으면 한달치만 가져오는 듯

today = datetime.today().date() 
print(today)
symbol = "MMM"
data = yf.Ticker(symbol).history(end=today)
print(data['Close'])

2025-07-01
Date
2025-06-02 00:00:00-04:00    146.399994
2025-06-03 00:00:00-04:00    148.130005
2025-06-04 00:00:00-04:00    146.809998
2025-06-05 00:00:00-04:00    146.259995
2025-06-06 00:00:00-04:00    145.500000
2025-06-09 00:00:00-04:00    144.229996
2025-06-10 00:00:00-04:00    145.059998
2025-06-11 00:00:00-04:00    147.179993
2025-06-12 00:00:00-04:00    144.770004
2025-06-13 00:00:00-04:00    142.320007
2025-06-16 00:00:00-04:00    145.070007
2025-06-17 00:00:00-04:00    142.210007
2025-06-18 00:00:00-04:00    142.509995
2025-06-20 00:00:00-04:00    144.029999
2025-06-23 00:00:00-04:00    147.600006
2025-06-24 00:00:00-04:00    149.869995
2025-06-25 00:00:00-04:00    147.750000
2025-06-26 00:00:00-04:00    150.759995
2025-06-27 00:00:00-04:00    152.020004
2025-06-30 00:00:00-04:00    152.240005
Name: Close, dtype: float64


In [2]:
def one_day_mlp_price(symbol: str, target_date: str) -> float:

    from_date = datetime.strptime(target_date, "%Y-%m-%d")
    # print(f"from_date : {from_date}")
    start_dt = from_date - timedelta(days=150)
    # print(f"start_dt : {start_dt}")

    df = yf.download(symbol, start=start_dt.strftime("%Y-%m-%d"), end=from_date.strftime("%Y-%m-%d"))
    
    if isinstance(df.columns, pd.MultiIndex):
        df = df.xs(symbol, axis=1, level=1)

    df.dropna(subset=['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)

    # 2. 모델 및 스케일러 불러오기
    model = tf.keras.models.load_model('MLP_model_3.h5', compile=False)
    X_scaler = joblib.load('MLP_X_scaler_3.joblib')
    y_scaler = joblib.load('MLP_y_scaler_3.joblib')

    # 3. LSTM, CNN 모델로부터 예측값 얻기
    Close = lstm_model(target_date, symbol, df)
    Label = cnn_model(target_date, symbol, df)

    # 4. 예측할 새 데이터 구성
    Yes_Close = df['Close'].iloc[-1]  # 장 안 열렸으니 오늘 얻을 수 있는 최신 종가 넣어줘야함 그래야 에측할 actual_close를 얻을 수 있다
    
    features_to_scale = np.array([[Close, Yes_Close]])
    features_scaled = X_scaler.transform(features_to_scale)

    # 6. Label은 정규화 없이 원본 그대로 넣기
    input_for_model = np.hstack([features_scaled, np.array([[Label]])])  # shape (1,3)

    # 7. 예측 및 역변환
    prediction_scaled = model.predict(input_for_model)
    prediction = y_scaler.inverse_transform(prediction_scaled)

    print(f"예측된 최종 종가값: {prediction[0][0]:.2f}")
    return float(prediction[0][0])

In [3]:
# 하루 치 예측 저장
# 25일꺼 예측했음

import csv
import os
from datetime import datetime

target_date = datetime.today().strftime("%Y-%m-%d")
input_file = "completed_symbols_h5.txt"
output_file = "oneDay_predictedPrices.csv"
# output_dir = "oneDay_history"
# output_file = os.path.join(output_dir, "oneDay_predictedPrices.csv")

# 종목 리스트 불러오기
with open(input_file, "r") as f:
    symbols = [line.strip() for line in f if line.strip()]

# 기존에 있는 심볼을 먼저 읽어둠 (중복 저장 방지)
existing_symbols = set()
if os.path.exists(output_file):
    with open(output_file, "r", newline="") as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None)  # 헤더 스킵
        for row in reader:
            if row:
                existing_symbols.add(row[0])

# CSV 파일을 append 모드로 열기
with open(output_file, "a", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # 파일이 비어 있으면 헤더 추가
    if os.path.getsize(output_file) == 0:
        writer.writerow(["Symbol", "PredictedPrice"])
        csvfile.flush()

    for symbol in symbols:
        if symbol in existing_symbols:
            print(f"[SKIP] 이미 존재하는 종목: {symbol}")
            continue
        try:
            price = one_day_mlp_price(symbol, target_date)
            writer.writerow([symbol, price])
            csvfile.flush()
            print(f"[SUCCESS] {symbol}: {price}")
        except Exception as e:
            print(f"[ERROR] {symbol}: {e}")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
[CNN_PY : period] 82 <= df_slice < 102
[CNN_PY : period] 2025-06-02 00:00:00 <= df_slice <= 2025-06-30 00:00:00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
예측된 최종 종가값: 152.01
[SUCCESS] MMM: 152.01376342773438


c:\Users\pyeon\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
[CNN_PY : period] 82 <= df_slice < 102
[CNN_PY : period] 2025-06-02 00:00:00 <= df_slice <= 2025-06-30 00:00:00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
예측된 최종 종가값: 65.11
[SUCCESS] AOS: 65.11444854736328


[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
[CNN_PY : period] 82 <= df_slice < 102
[CNN_PY : period] 2025-06-02 00:00:00 <= df_slice <= 2025-06-30 00:00:00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
예측된 최종 종가값: 136.38
[SUCCESS] ABT: 136.37965393066406


c:\Users\pyeon\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
[*********************100%***********************]  1 of 1 completed


KeyboardInterrupt: 

In [7]:
# 다음날 점심쯤에 이 file_path로 csv 저장하면 됨. 즉 한국 22일에 돌리면 21일.csv로 저장함

import pytz
import datetime

symbol = "AAPL"
today_kor = datetime.datetime.now(pytz.timezone("ROK")).date()

# 가장 최근 거래일 추출
sample_df = yf.download(symbol, end=today_kor)
if sample_df.empty:
    raise RuntimeError("기준 종목의 데이터가 없습니다.")

last_date = sample_df.index[-1].date()
file_path = f'oneDay_{last_date.strftime("%y%m%d")  }.csv'

print(file_path)

[*********************100%***********************]  1 of 1 completed

oneDay_250624.csv


In [9]:
# 실제 가격 호출 후 diff 까지 수행

import pandas as pd
import yfinance as yf
import datetime
import os
# from datetime import datetime, timedelta

# 오늘 날짜 설정
today = datetime.datetime.now().date()
today_str = today.strftime('%Y-%m-%d')

# 파일명 : 위의 셀 코드로 대체
# file_path = 'oneDay_{date}.csv'

# 기존 CSV 불러오기 또는 초기화
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
else:
    df = pd.read_csv('oneDay_predictedPrices.csv')
    df['actualPrice'] = None
    df['diff'] = None
    df.to_csv(file_path, index=False)

# 처리 반복
for i, row in df.iterrows():
    symbol = row['Symbol']
    if pd.notna(row['actualPrice']):
        print(f"{symbol}: 이미 처리됨, 건너뜀")
        continue

    try:
        stock = yf.Ticker(symbol)
        data = stock.history(start = today - datetime.timedelta(days=7), end = today)
        if not data.empty:
            actual_price = data['Close'].iloc[-1] # -1로 바꾸는게 나음
            predicted_price = row['PredictedPrice']
            diff_percent = abs(predicted_price - actual_price) / actual_price * 100

            df.at[i, 'actualPrice'] = actual_price
            df.at[i, 'diff'] = diff_percent

            print(f"{symbol}: Predicted={predicted_price:.2f}, Actual={actual_price:.2f}, Diff={diff_percent:.2f}%")
        else:
            print(f"{symbol}: 오늘자 종가 데이터 없음")
    except Exception as e:
        print(f"{symbol}: 오류 발생 - {e}")

    # 변경 즉시 저장
    df.to_csv(file_path, index=False)

# 전체 평균 오차율 출력
valid_diffs = df['diff'].dropna()
if not valid_diffs.empty:
    average_diff = valid_diffs.mean()
    print(f"\n전체 평균 예측 오차율: {average_diff:.2f}%")
else:
    print("\n아직 유효한 diff 데이터가 없습니다.")


MMM: Predicted=148.21, Actual=149.87, Diff=1.11%
AOS: Predicted=63.52, Actual=64.94, Diff=2.18%
ABT: Predicted=132.54, Actual=138.08, Diff=4.01%
ABBV: Predicted=183.04, Actual=185.55, Diff=1.35%
ACN: Predicted=300.69, Actual=300.72, Diff=0.01%
ADBE: Predicted=385.74, Actual=382.34, Diff=0.89%
AMD: Predicted=129.84, Actual=138.43, Diff=6.20%
AES: Predicted=10.28, Actual=10.47, Diff=1.86%
AFL: Predicted=104.64, Actual=104.16, Diff=0.46%
A: Predicted=117.31, Actual=117.64, Diff=0.28%
APD: Predicted=277.09, Actual=281.00, Diff=1.39%
ABNB: Predicted=132.02, Actual=132.15, Diff=0.10%
AKAM: Predicted=79.14, Actual=80.06, Diff=1.15%
ALB: Predicted=56.62, Actual=60.18, Diff=5.92%
ARE: Predicted=73.16, Actual=75.37, Diff=2.93%
ALGN: Predicted=185.77, Actual=186.58, Diff=0.43%
ALLE: Predicted=141.63, Actual=143.61, Diff=1.38%
LNT: Predicted=61.13, Actual=61.16, Diff=0.05%
ALL: Predicted=197.72, Actual=196.05, Diff=0.85%
GOOGL: Predicted=166.52, Actual=166.77, Diff=0.15%
GOOG: Predicted=167.08, Ac